In [ ]:
!pip install transformers
!pip install transformers[sentencepiece]
!pip install xformers

In [9]:
from transformers import pipeline
import pandas as pd
import numpy as np
import os, re
from transformers import AutoTokenizer

In [10]:
SOURCE_PATH = '/content/test_ololo (2).xlsx' #исходник с темами и описанием
OUTPUT_PATH = '/content/result.csv' #файл с сортированными темами
THEME_NAMES_PATH = '/content/themes.xlsx' #файл с темами (категориями)
SUMMARY = '/content/summary.csv' #файл подсчет количества тем

def cleaning_up(raw_text):
    regex1 = r"{[a-zA-z0-9.,!|;:*#]+}|![a-zA-z0-9.*,!|;:#]+!|\n|\xa0"
    regex2 =r"[Дд]обр.{1,3}\s.{1,6}[!.\s]|[зЗ]драв.{1,8}[!.\s]|[;]"
    subst = ''
    result = re.sub(regex1, subst, raw_text, 0, re.MULTILINE)
    result = re.sub(regex2, subst, result, 0, re.MULTILINE)
    result = result.split('УВЕДОМЛЕНИЕ О КОНФИДЕНЦИАЛЬНОСТИ')[0].strip()

    return result

def load_and_process():
    #Объединяем столбцы в одни текст
    print('Обработка исходных данных')
    data = pd.read_excel(SOURCE_PATH, header=0)
    data = data.iloc[:,:3]
    data = data.fillna('')
    data['Описание'] = data['Описание'].apply(lambda row: cleaning_up(row)) #вычищаем весь мусор

    data['Разделитель'] = '. '
    data['Тема_с_описанием'] = data['Тема'] + data['Разделитель'] + data['Описание'] #объединяем тему с описанием
    data = data[['Тема', 'Тема_с_описанием']]

    return data



In [ ]:
#Подгрузка предобученной модели ИИ (https://huggingface.co/models?pipeline_tag=zero-shot-classification&language=ru&sort=trending)
print('Загрузка предобученной нейронной сети')
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', truncation=True, truncation_side='right')
classifier = pipeline("text-classification", model='SIA86/bert-cased-text_class', tokenizer=tokenizer)


In [ ]:
print('Загрузка и очистка входных данных')
preprocessed_data = load_and_process()
#preprocessed_data

In [ ]:
#Чтение данных и запись тем в новый файл. Если чтение прервется - просто перезапустить скрипт. Чтения начнется с того места, где остановились
print('Проверка наличия ранее созданного файла для вывода данных')
if not os.path.isfile(OUTPUT_PATH): #если файл не был ранее создан
    print('Файл не найден')
    with open(OUTPUT_PATH, 'w', encoding="utf-8") as fw:
        fw.write('Тема;Тема_с_описанием;Категория;\n') #создаем файл и записываем туда заголовок
        print(f'Создан файл для вывовда данных: {OUTPUT_PATH}')
else:
    print(f'Найден ранее созданный файл {OUTPUT_PATH}')

with open(OUTPUT_PATH, 'r', encoding="utf-8") as fr: #определяем сколько строк уже проанализировано ранее
    lines = fr.readlines()
    total_length = len(lines)-1 #всего строк в файле за вычетом заголовка
    print(f'Всего записей в файле для вывода данных: {total_length}')

cathegory = pd.read_excel(THEME_NAMES_PATH, header=None).to_dict()[0]

In [ ]:
for row in preprocessed_data[total_length:].rolling(1): #двигаемся по строкам начинная с места остановки
  print(f'ИИ анализирует {row.index[0]} строку из таблицы')
  result = classifier(row['Тема_с_описанием'].iloc[0]) #отправляем строку к ИИ
  indx = int(result[0]['label'].removeprefix('LABEL_'))

  with open(OUTPUT_PATH, 'a', encoding="utf-8") as fw:
    fw.write(f"{row['Тема'].iloc[0]};{row['Тема_с_описанием'].iloc[0]};{cathegory[indx]}")
    fw.write('\n')

print('Анализ исходных данных завершен.')



In [28]:
#сохранение в excel формат
df = pd.read_csv(OUTPUT_PATH, sep=';')
df.to_excel(OUTPUT_PATH.removesuffix('csv')+'xlsx', header=False)

In [ ]:
#Считаем количество тем и выводим результат в отдельный файл
print('Идет подсчет совпадений по каждой теме')
themes_df = pd.read_csv(OUTPUT_PATH, sep=';')
themes_df = themes_df['Категория']

df = pd.DataFrame(np.zeros(shape=(1,len(cathegory))), columns=list(cathegory.values()))

for cathegory in list(cathegory.values()):
    try:
        df[cathegory] = themes_df.value_counts()[cathegory]
    except KeyError:
        pass
df= df.T
df.index = df.index.rename('Тема')
df = df.rename(columns={0:'Количество'})
df.to_csv(SUMMARY) #запись в CSV файл
print(f'Статистика по темам выведена в файл {SUMMARY}')
